# 1. 下载IMDB公开数据集，并处理

In [23]:
import os

imdb_dir = "./aclImdb"
train_dir = os.path.join(imdb_dir, "train")


labels = []
texts = []

for label_type in ["neg", "pos"]:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname [-4:] == ".txt":
            f = open(os.path.join(dir_name, fname), encoding='utf-8')
            texts.append(f.read())
            f.close()
            if label_type == "neg":
                labels.append(0)
            else:
                labels.append(1)                    

# 2. 制作训练集和验证集

In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences
import numpy as np

maxlen = 100 # cuts off review after 100 words
training_samples = 200 # Trains on 200 samples
validation_samples = 10000 # Validates o 10000 samples
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index                   # Length: 88582
print("Found %s unique tokens." % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print("Shape of data tensor:", data.shape)
print("Shape of label tensor:", labels.shape)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples] # (200, 100)
y_train = labels[:training_samples] # shape (200,)
x_val = data[training_samples:training_samples+validation_samples] # shape (10000, 100)
y_val = labels[training_samples:training_samples+validation_samples] # shape (10000,)


Using TensorFlow backend.


Found 88582 unique tokens.
Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


# 3. 下载glove词向量，并读入

In [25]:
glove_dir = "./"

embeddings_index = {}

f = open(os.path.join(glove_dir, "glove.6B.50d.txt"), encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


# 4. 读入词向量

In [26]:
embedding_dim = 50 # GloVe contains 50-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 50)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

# 5. 搭建模型

In [27]:
model = keras.models.Sequential() 
model.add(keras.layers.Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences = True)))
model.add(keras.layers.GlobalMaxPool1D())
model.add(keras.layers.Dense(16, activation = "relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          58880     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 560,961
Trainable params: 560,961
Non-trainable params: 0
_________________________________________________________________


# 6. 加载预训词向量

In [28]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False 

# 7. 训练

In [29]:
model.compile(optimizer = "rmsprop", 
              loss = "binary_crossentropy", # in a multiclass problem categorical_crossentropy would be used
              metrics = ["acc"]) 
history = model.fit(x_train, y_train,
                   epochs = 10,
                   batch_size = 32,
                   validation_data = (x_val, y_val))
model.save_weights("pre_trained_glove_model.h5")  

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 6s 31ms/sample - loss: 0.6943 - acc: 0.5550 - val_loss: 0.7011 - val_acc: 0.4923
Epoch 2/10
200/200 [==============================] - 3s 16ms/sample - loss: 0.6654 - acc: 0.5850 - val_loss: 0.6785 - val_acc: 0.6087
Epoch 3/10
200/200 [==============================] - 3s 16ms/sample - loss: 0.6481 - acc: 0.6850 - val_loss: 0.7081 - val_acc: 0.4925
Epoch 4/10
200/200 [==============================] - 3s 17ms/sample - loss: 0.6363 - acc: 0.6700 - val_loss: 0.7292 - val_acc: 0.4923
Epoch 5/10
200/200 [==============================] - 3s 17ms/sample - loss: 0.6148 - acc: 0.6800 - val_loss: 0.6750 - val_acc: 0.5296
Epoch 6/10
200/200 [==============================] - 3s 17ms/sample - loss: 0.6119 - acc: 0.7100 - val_loss: 0.6509 - val_acc: 0.6468
Epoch 7/10
200/200 [==============================] - 3s 17ms/sample - loss: 0.5820 - acc: 0.7600 - val_loss: 0.6450 - val_acc: 0.6489
Epoch 8